In [10]:
import time
import sys
from collections import Counter
import pandas as pd
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
%pylab inline
import pylab
import gzip
from Bio import SeqIO
import seaborn as sns
import string
from matplotlib import rcParams

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [11]:
#Open .txt files for mating data
#If your files are in fastq, just change the 
f1 = open('/Users/dyounger/GoogleDrive/UW_Work/PAPER_YSA/Data/NGS/NGS_Mating_Characterization/SSM_Libraries/BclXL_SSM_R1.txt', "r")
f2 = open('/Users/dyounger/GoogleDrive/UW_Work/PAPER_YSA/Data/NGS/NGS_Mating_Characterization/SSM_Libraries/BclXL_SSM_R2.txt', "r")
#Open .csv file for matching SSM mutation to barcode
f3 = pd.read_csv('/Users/dyounger/GoogleDrive/UW_Work/PAPER_YSA/Code/SSMLibraryAnalysis/CSV_outputs_from_step1/XCDP07_BarcodeLibrary.csv')
#open .csv file for residue types
f5 = pd.read_csv('/Users/dyounger/GoogleDrive/UW_Work/PAPER_YSA/Data/XCDP07_residue_type.csv')

#specify list of all targets
all_targets=['Bfl1','BclB','BclXL','Bcl2','BclW','Mcl1']
#specify intended target for binder
target='BclXL'

In [12]:
#process the text files to get them into the desired format
z1 = f1.readline()
a1=[]
x=1
while True:
    a1.append(f1.readline())
    z1 = f1.readline()
    z1 = f1.readline()
    z1 = f1.readline()
    if z1.strip() == "":
        break
    x=x+1
f1.close()
df=pd.DataFrame(data=a1,columns=['read1'])

z2 = f2.readline()
a2=[]
x=1
while True:
    a2.append(f2.readline())

    z2 = f2.readline()
    z2 = f2.readline()
    z2 = f2.readline()
    if z2.strip() == "":
        break
    x=x+1
f2.close()
read2df=pd.DataFrame(data=a2,columns=['read2'])
df=df.join(read2df)

#process the barcodes/mutations file to get it into the desired format->dictionary
mutations = pd.DataFrame(data=f3['barcode'], columns = ['barcode'])
temp = pd.DataFrame(data=f3['combined'], columns = ['combined'])
temp2 = pd.DataFrame(data=f3['position'], columns = ['position'])
temp3 = pd.DataFrame(data=f3['mutation'], columns = ['mutation'])
temp4 = pd.DataFrame(data=f3['group_size'], columns = ['group_size'])
mutations = mutations.join(temp)
mutations = mutations.join(temp2)
mutations = mutations.join(temp3)
mutations = mutations.join(temp4)
BC_dictionary = mutations.set_index('barcode').T.to_dict('list')

In [13]:
# Filter for read quality by looking at conserved regions
df=df[df.read1.str.slice(27,37) == 'GAAAAGCGGC']
df=df[df.read1.str.slice(52,62) == 'AATAAAGGAG']
df=df[df.read2.str.slice(27,37) == 'CCAGGTATCG']
df=df[df.read2.str.slice(52,62) == 'TTATGTCACG']

# Grab just the barcodes and join them into one dataframe
bc=pd.DataFrame(data=df.read1.str.slice(37,47),columns=['read1'])
read2bc=pd.DataFrame(data=df.read2.str.slice(37,47),columns=['read2'])
bc=bc.join(read2bc)
bc.columns=['MATa','barcode']
print len(bc)

1372458


In [14]:
#Replace MATa barcodes with the target name
bc = bc.replace(['TCTGAATCAA','TTATTACCAT','GGTTCTATAA','AGTAGATCGT','CTCACGTGTG','AATCCAACGA'],['Bcl2','BclB','BclXL','Bfl1','BclW','Mcl1'])
# Filter for correct barcodes only
bc=bc[(bc.MATa == 'Bfl1') | (bc.MATa=='BclB') | (bc.MATa=='Bcl2') | (bc.MATa=='BclXL') | (bc.MATa=='BclW') | (bc.MATa=='Mcl1')]
#make list of non-targets:
others=all_targets
others.remove(target)

In [15]:
#For each barcode, count the number of times it is seen with a given MATa
bc1=bc.groupby('barcode').MATa.apply(Counter).unstack().reset_index().fillna(value=0)

In [16]:
#Replace MATalpha barcodes with the mutation from the dictionary
per_barcode = pd.merge(bc1,mutations,on='barcode',how='right').fillna(value=0)

In [17]:
#combine all barcodes that have the same mutation : sum counts
per_mutation = per_barcode.drop('barcode',1)
per_mutation = per_mutation.groupby('combined').agg({'Bcl2' : np.sum,
                                                     'BclB' : np.sum,
                                                     'BclW' : np.sum,
                                                     'BclXL' : np.sum,
                                                     'Bfl1' : np.sum,
                                                     'Mcl1' : np.sum,
                                                     'group_size' : np.sum,
                                                     'mutation' : np.any,
                                                     'position' : np.mean}).reset_index()
#add column to each dataframe with total counts
per_mutation['count']=per_mutation['Bcl2']+per_mutation['BclB']+per_mutation['BclW']+per_mutation['BclXL']+per_mutation['Bfl1']+per_mutation['Mcl1']
per_mutation.columns = ['combined','naive','Bfl1','Mcl1','position','BclW','BclB','BclXL','aa','Bcl2','total']

In [19]:
#designate each position as core, surface, interface
f5['position'] = f5['position'].astype(float)
per_mutation=pd.merge(per_mutation,f5,on='position',how='left')

In [21]:
#sort and reset index
per_mutation=per_mutation.sort_values(['position','aa'],ascending=[True,True]).reset_index().fillna(value=0)
del per_mutation['index']
#reorder columns
df_summary=per_mutation[['combined','aa','position','naive','total','Bfl1','BclB','Bcl2','BclW','BclXL','Mcl1','type']]

In [23]:
#Save the dataframe!
df_summary.to_csv('XCDP07_df_summary'+'.csv')